In [ ]:
import glob
from concurrent import futures

import isambard
import numpy

<h2>Modelling of Natural Parallel Coiled Coils Using ISAMBARD</h2>

To test the structure building apet of ISAMBARD, natural structures will be recreated using the `CoiledCoil` topology. There are multiple steps that are required to do this:

1. Find coiled-coil regions in natural structures.
    * This is performed using data extracted from [CC+](http://coiledcoils.chm.bris.ac.uk/ccplus/search/).
1. Translate the register into phica values.
1. Setup a database to store the results.
1. Create the reference structure.
1. Initialise and run the optimiser.

<h3>1. Prepare CC Records</h3>

In [ ]:
header = "| pdb  | coiledcoil | helices | orientation | partnering | repeat    | docktype | minimum | average | maximum | redundancy | pdb  | coiledcoil | helix | chain | helixstart | helixlength | helixend | helixsequence                                       | assignedstart | assignedlength | assignedend | assignedsequence              | assignedregister              | assignedinterrupts |"

In [ ]:
header = [x.strip() for x in header.split('|')][1:-1]

In [ ]:
header

In [ ]:
with open('all_coiled_coils.txt', 'r') as inf:
    cc_records = inf.readlines()
cc_records = [x.strip().split() for x in cc_records]

Filter for cannonical with redundency of 70.

In [ ]:
def parallel_filter(record):
    if record[5] != 'canonical':
        return False
    if int(record[10]) > 70:
        return False
    if record[4] != 'homo':
        return False
    orientations = [x for x in record[3] if (x == 'u') or (x == 'd')]
    if len(orientations) != 3:
        return False
    if len(set(orientations)) != 1:
        return False
    return True

In [ ]:
parallel_trimers = list(filter(parallel_filter, cc_records))

In [ ]:
trimer_ccs = {}
for helix in parallel_trimers:
    k = tuple(helix[:2])
    if k in trimer_ccs:
        trimer_ccs[k].append(helix)
    else:
        trimer_ccs[k] = [helix]

In [ ]:
assert all([True if len(x) == 3 else False for x in trimer_ccs.values()])

In [ ]:
final_trimer_records = []
for helices in trimer_ccs.values():
    record = []
    for i in range(len(helices[0])):
        record.append([helix[i] for helix in helices])
    final_trimer_records.append(record)

In [ ]:
len(final_trimer_records)

<h3>2. Translate Register to $\phi C \alpha$</h3>

In [ ]:
register_phis = {
    'a': 25.714285714285715,
    'e': 77.14285714285714,
    'b': 128.57142857142858,
    'f': 180.0,
    'c': 231.42857142857144,
    'g': 282.8571428571429,
    'd': 334.2857142857143
}

<h3>3. Setup Results Database</h3>

Using SQLite to store the data from the rebuild to make it easier to transport and query.

In [ ]:
from sqlalchemy import create_engine, Column, Float, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Connect to results database
engine = create_engine('sqlite:///natural_rebuild.db', echo=False)
Base = declarative_base()


class CCPRebuildRes(Base):
    __tablename__ = 'parallel_trimer_rebuild'

    id = Column(Integer, primary_key=True)
    pdb = Column(String)
    cc_id = Column(Integer)
    oligomer_state = Column(Integer)
    sequence = Column(String)
    start1 = Column(Integer)
    end1 = Column(Integer)
    start2 = Column(Integer)
    end2 = Column(Integer)
    start3 = Column(Integer)
    end3 = Column(Integer)
    chain1 = Column(String)
    chain2 = Column(String)
    chain3 = Column(String)
    register = Column(String)
    radius = Column(Float)
    pitch = Column(Float)
    phica = Column(Float)
    aa_rmsd = Column(Float)
    bb_rmsd = Column(Float)
    ca_rmsd = Column(Float)
    aa_rmsd_100 = Column(Float)
    bb_rmsd_100 = Column(Float)
    ca_rmsd_100 = Column(Float)

    def __repr__(self):
        return "<Coiled Coil Rebuild (cc_id='{}', CA_RMSD='{})>".format(
            self.cc_id, self.ca_rmsd)


Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
rebuild_res_session = Session()

<h3>4. Create the Reference Structure</h3>

In [ ]:
def create_reference_ampal(pdb, cc, sequence, starts, ends, chains):
    base_mmol_path = '/structural_bioinformatics/data/{}/{}/structures/*'
    mmol_path = glob.glob(base_mmol_path.format(pdb[1:3], pdb[:4]))[0]
    reference_cc_full = isambard.ampal.convert_pdb_to_ampal(mmol_path)
    return reference_cc_full

<h3>5. Create Function for Rebuilding and Logging</h3>

In [ ]:
def rmsd_100(rmsd, n):
    return rmsd/(1+numpy.log(numpy.sqrt(n/100)))

In [ ]:
def align_from_record(record, gens=50):
    pdb = record[0][0]
    cc_id = record[1][0]
    sequence = record[20][0]
    starts = record[17]
    ends = record[19]
    chains = record[12]
    register = record[21][0]
    sequence = record[20][0]
    hel_len = len(sequence)
    n = len(chains)
    
    try:
        reference = create_reference_ampal(pdb, cc_id, sequence, starts, ends, chains)
    except KeyError:
        return pdb, cc_id
    opt = isambard.optimisation.DE_RMSD(
        isambard.specifications.CoiledCoil.from_parameters, reference.pdb)
    opt.parameters([sequence]*n,
              [6.0, 200, register_phis[register[0]]],
              [1.0, 150, 20],
              [n, hel_len, 'var0', 'var1', 'var2'])
    opt.run_opt(20, gens, 1)
    parameters = opt.parse_individual(opt.halloffame[0])
    cc = isambard.specifications.CoiledCoil.from_parameters(*parameters)
    cc.pack_new_sequences([sequence]*n)
    rmsds = isambard.external_programs.run_profit(
        cc.pdb, reference.pdb, path1=False, path2=False)
    rmsd_100s = [rmsd_100(x, n*hel_len) for x in rmsds]
    rbr = CCPRebuildRes(pdb=pdb, cc_id=int(cc_id), oligomer_state=n, sequence=sequence,
                        start1=int(starts[0]), end1=int(ends[0]),
                        start2=int(starts[1]), end2=int(ends[1]),
                        chain1=chains[0], chain2=chains[1], register=register[0],
                        radius=parameters[2], pitch=parameters[3], phica=parameters[4],
                        aa_rmsd=rmsds[2], bb_rmsd=rmsds[1], ca_rmsd=rmsds[0],
                        aa_rmsd_100=rmsd_100s[2], bb_rmsd_100=rmsd_100s[1],
                        ca_rmsd_100=rmsd_100s[0])
    rebuild_res_session.add(rbr)
    rebuild_res_session.commit()
    return

<h3>6. Run with Parallelisation</h3>

In [ ]:
%%capture
with futures.ProcessPoolExecutor(max_workers=24) as executor:
    failures = executor.map(align_from_record, final_trimer_records)

In [ ]:
failed = []
for x in failures:
    try:
        failed.append(x)
    except:
        pass

In [ ]:
failed